[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MeteoSwiss/opendata-nwp-demos/blob/main/01_retrieve_process_precip.ipynb)

# Retrieving, Processing, and Visualizing Deterministic Precipitation ICON-CH2-EPS Forecast Data

This notebook demonstrates the full workflow for accessing, processing, and visualizing determinstic ICON-CH2-EPS precipitation forecast data. The data is provided by MeteoSwiss as part of Switzerland’s [Open Government Data (OGD) initiative](https://www.meteoswiss.admin.ch/services-and-publications/service/open-data.html).

The core functionality is powered by the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — developed by MeteoSwiss to simplify working with numerical weather model data. It includes the `ogd_api` module for data access, along with utilities for applying temporal and spatial operators to model output, such as deaccumulation and regridding.

For visualization, this notebook uses the [earthkit-plots](https://earthkit-plots.readthedocs.io/en/latest/examples/guide/01-introduction.html) library developed by ECMWF, which offers intuitive plotting tools for meteorological data.

---

## 🔍 **What You’ll Do in This Notebook**

 🛰️  **Retrieve**  
    Fetch deterministic ICON-CH2-EPS forecast data (e.g., total precipitation) via the `ogd_api` module.

 🧪  **Deaggregate**  
    Convert precipitation accumulated over the total forecast duration into hourly values using [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/)'s `time_operators` module.

 🧭  **Regrid**  
    Interpolate ICON-CH2-EPS data from its native, icosahedral grid to the regular latitude/longitude grid [WGS84 (EPSG:4326)](https://epsg.io/4326).

 🌍  **Visualize**  
    Plot the processed data on a map using [earthkit-plots](https://earthkit-plots.readthedocs.io/en/latest/examples/guide/01-introduction.html).

---

## Retrieving Precipitation Forecast
In this first part, we retrieve deterministic ICON-CH2-EPS precipitation forecast data. To access this data, we use the `ogd_api` module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset), which provides structured access to Switzerland’s open geospatial data.

#### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON-CH1/2-EPS forecast datasets interactively before writing code, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.


![browser-ch2.png](./images/browser-ch2.png)

> ⚙️ **Google Colab Setup**  
> This cell installs all required dependencies if you're running the notebook in **Google Colab**.  
> It is automatically skipped when running in a local Jupyter environment.

In [1]:
# 📦 Google Colab Setup (skipped if not running in Colab)
import sys

def is_colab():
    return "google.colab" in sys.modules

if is_colab():
    !git clone https://github.com/MeteoSwiss/opendata-nwp-demos.git
    %cd opendata-nwp-demos
    !pip install poetry && poetry config virtualenvs.in-project true && poetry install --no-ansi
    import sys, os, pathlib
    venv = pathlib.Path(".venv")
    site = venv / "lib" / f"python{sys.version_info.major}.{sys.version_info.minor}" / "site-packages"
    sys.path.insert(0, str(site))
    os.environ["ECCODES_DEFINITION_PATH"] = str((venv / "share/eccodes-cosmo-resources/definitions").resolve())

### Creating Requests
To retrieve the forecast data, we first define an API request using the `ogd_api.Request` class.
The request targets multiple lead times, which is the elapsed time since the forecast's start time (reference or initialization datetime).
In this example, we retrieve hourly precipitation accumulations (`TOT_PREC`) for three consecutive lead times: +0h, +1h and +2h. 

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

In [2]:
from meteodatalab import ogd_api

# Define target lead times +0h, +1h and +2h
lead_times = ["P0DT0H", "P0DT1H", "P0DT2H"]

req = ogd_api.Request(
    collection="ogd-forecasting-icon-ch2",
    variable="TOT_PREC",
    ref_time="latest",
    perturbed=False,
    lead_time=lead_times,
)

Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch2`). |
| `variable`           | Meteorological variable of interest (`TOT_PREC` = total precipitation). |
| `ref_time` | Initialization time of the forecast in **UTC**, provided as either:<br>- The string `"latest"` to select the most recent available run (`ref_time`) for each requested asset. Be cautious: since each asset covers one model, ref_time, lead time, and variable, multiple requests during data publication may return assets with different `ref_time` values. <br>- [datetime.datetime](https://docs.python.org/3/library/datetime.html#datetime-objects) object (e.g.,<br> &nbsp; `datetime.datetime(2025, 5, 22, 9, 0, 0, tzinfo=datetime.timezone.utc)`) <br>- [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Combined_date_and_time_representations) date string (e.g., `"2025-05-22T09:00:00Z"`)|
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `lead_time`            | Forecast lead time, provided as either:<br>– [datetime.timedelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) object (e.g., `datetime.timedelta(hours=2)`) <br>– [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Durations) duration string (e.g., `"P0DT2H"`)|

### Retrieving Data
We now send our request to the API and retrieve the resulting dataset using the `get_from_ogd()` function.
The response is returned as an **[xarray.DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html)**, which is efficient for handling multi-dimensional data.

> 💡 **Tip**: Use temporary caching with earthkit-data to skip repeated downloads — it's auto-cleaned after the session.
> *For more details, see the [earthkit-data caching docs](https://earthkit-data.readthedocs.io/en/latest/examples/cache.html)*.

> 💡 **Hint**: If you get an error message containing `HTTPError: 403 Client Error: Forbidden for url`, you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [3]:
from earthkit.data import config
config.set("cache-policy", "temporary")

tot_prec = ogd_api.get_from_ogd(req)

The resulting `xarray.DataArray` has the following dimensions:

- `eps` (ensemble members): 1 member (for deterministic data)
- `ref_time`: single reference time (the latest available run)
- `lead_time`: 3 hourly intervals
- `cell`: 283,876 spatial grid points

It includes attributes specifying some meta data such as:

- `parameter`: information on the meteorogical parameter (e.g. name, units etc.)
- `vcoord_type`: information of the vertical coordinate type, here `surface`

> &#x1F525; **Expert users:** If you are interested in detailed meta data information, i.e. the GRIB encodings, you can retrieve it from the `metadata` attribute.
Please refer to the [earthkit-data metadata documentation](https://earthkit-data.readthedocs.io/en/latest/examples/metadata.html).

In [4]:
tot_prec

<xarray.DataArray (eps: 1, ref_time: 1, lead_time: 3, cell: 283876)> Size: 3MB
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]], dtype=float32)
Coordinates:
  * eps         (eps) int64 8B 0
  * ref_time    (ref_time) datetime64[ns] 8B 2025-06-18T06:00:00
  * lead_time   (lead_time) timedelta64[ns] 24B 00:00:00 01:00:00 02:00:00
    lon         (cell) float32 1MB -0.1544 -0.1637 -0.1915 ... 12.82 12.83 12.81
    lat         (cell) float32 1MB 46.97 46.98 46.98 47.13 ... 50.08 50.06 50.05
    valid_time  (ref_time, lead_time) datetime64[ns] 24B 2025-06-18T06:00:00 ...
Dimensions without coordinates: cell
Attributes:
    metadata:     <earthkit.data.readers.grib.metadata.StandAloneGribMetadata...
    parameter:    {'centre': 'lssw', 'paramId': 500041, 'shortName': 'TOT_PRE...
    geography:    {'gridType': 'unstructured_grid', 'bitmapPresent': 0}
    vref:         geo
    vcoord_type:  surface
    origin_z:     0.0

## Data Preprocessing

Once the data has been loaded, continue with the preprocessing of the data if needed. The following preprocessing steps
are described here:
- Disaggregating the accumulated precipitation data
- Regridding the data to the regular latitude/longitude grid [WGS84 (EPSG:4326)](https://epsg.io/4326)

### Disaggregating Precipitation
The precipitation data is aggregated from the forecast reference time, meaning each hourly forecast value represents the amount of precipitation from hour 0 up to the given lead time. To convert precipitation accumulated over the whole forecast time into one-hour precipitation, we use the `delta()` function provided by the `time_operators` module in [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/).

In [5]:
from meteodatalab.operators import time_operators as time_ops
import numpy as np

prec_1h = time_ops.delta(tot_prec, np.timedelta64(1,"h"))

For the next steps, we'll work with a **single lead time** from the forecast.
Xarray's `.isel()` method lets us easily select along a specific dimension — here, we use it to pick one lead time.


In [6]:
prec = prec_1h.isel(lead_time=2)

### Regridding to the Regular Grid [WGS84](https://epsg.io/4326)

To visualize or compare ICON-CH2-EPS model outputs with other datasets, it is often necessary to regrid the data from its native unstructured grid to a regular latitude/longitude grid.

This can be done using the `regrid` utilities provided by [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/).

We begin by defining a **target grid**, starting with its **spatial extent**, also known as the bounding box. This is specified by the minimum and maximum coordinate values along the x and y axes (commonly referred to as longitude and latitude when using a geographic CRS such as [WGS84 (EPSG:4326)](https://epsg.io/4326)). Note that in projected coordinate systems, these values are not technically “longitude” and “latitude,” but rather x and y coordinates in the chosen CRS.

In this example, we use the `regrid.iconremap()` function to perform the regridding. Depending on how the target grid's spatial extent is defined, it can fall into one of the following cases:
  - ✅ **Fully overlap** the ICON-CH1/CH2-EPS domain: all target grid points are interpolated.
  - ⚠️ **Partially overlap**: only the overlapping area will contain valid values; other points are filled with `NaN`.
  - ❌ **Lie completely outside** the ICON-CH1/CH2-EPS domain: all target grid values will be `NaN`.

📌 In our example, we define the spatial extent to fully match the **ICON-CH1/CH2-EPS** domain.

<p align="center">
  <img src="images/icon-ch1-ch2-eps-domain.png" alt="ICON domain illustration" width="450"/>
</p>

In [7]:
# Define the spatial extent of the target grid
# (we retain the original ICON-CH1/CH2-EPS domain)
xmin, xmax = -0.817, 18.183   # Longitude bounds
ymin, ymax = 41.183, 51.183   # Latitude bounds

Next, we define the **number of grid points** (`nx`, `ny`) to control the resolution of the regular grid.
    
To match the resolution of the ICON-CH2-EPS domain (≈ **2 km**), we define the number of points directly:


In [8]:
# Compute the number of grid points to match ~2 km resolution
nx, ny = 732, 557  # Number of grid points in x and y

> 🔢 [How to compute nx, ny for a custom resolution](./computing_nx_ny.md)

The parameters `nx` and `ny` are passed to `regrid.RegularGrid`, which defines the spatial geometry of the output grid.

In [9]:
from rasterio.crs import CRS
from meteodatalab.operators import regrid

# Create a regular lat/lon grid using EPSG:4326
destination = regrid.RegularGrid(
    CRS.from_string("epsg:4326"), nx, ny, xmin, xmax, ymin, ymax
)

Finally, remap the data to the regular grid.


In [10]:
prec_geo = regrid.iconremap(prec, destination)

## Visualizing Precipitation on a Map
We now visualize the 1-hour precipitation on a map. This uses [earthkit-plots](https://earthkit-plots.readthedocs.io/en/latest/examples/guide/01-introduction.html), the visualization component of [Earthkit](https://earthkit.readthedocs.io/en/latest/) — a suite of tools designed to simplify access, processing, and visualization of weather and climate data in Python.

In [11]:
import pandas as pd
import cartopy.crs as ccrs
from earthkit.plots.geo import bounds, domains
from earthkit.plots.styles import Style
import earthkit

from plot_utils.load_colormaps import load_ncl_rgb_colors
from plot_utils.plot_valid_data_frame import get_valid_data_frame


# === Setup Domain ===
bbox = bounds.BoundingBox(xmin, xmax, ymin, ymax, ccrs.Geodetic())
domain = domains.Domain.from_bbox(
    bbox=bbox,
    name="CH2"
)


# === Load custom colormap with levels ===
colors, levels = load_ncl_rgb_colors("precip_1h_11lev")


# === Create Map ===
chart = earthkit.plots.Map(domain=domain)
chart.grid_cells(prec_geo, x="lon", y="lat", style=Style(colors=colors, levels=levels))


# === Add Frame ===
frame_polygon = get_valid_data_frame(prec_geo)

if frame_polygon:
    x, y = frame_polygon.exterior.xy
    chart.ax.plot(x, y, color='black', linewidth=1, transform=ccrs.PlateCarree())
else:
    print("No valid frame polygon could be computed.")


# === Add Map Features ===
chart.land()
chart.coastlines()
chart.borders()
chart.gridlines()


# === Annotate Chart ===
ref_time = pd.to_datetime(prec_geo.coords["ref_time"].values[0]).strftime("%Y-%m-%d %H:%M UTC")
lead_time = prec_geo.coords["lead_time"].values.astype('timedelta64[h]')

parameter = prec_geo.attrs["parameter"]
title = f"Hourly Precipitation | {ref_time} (+{lead_time})"
legend_label = f"{parameter['name']} ({'mm'})"

chart.title(title)
chart.legend(label=legend_label)


# === Show Plot ===
chart.show()
